In [0]:
include __DIR__ . '/vendor/autoload.php';




In [1]:
$id_to_label  = array();
$id_to_number = array();
$id_count = 0;
$labels = explode("\n", file_get_contents("data/train_labels.csv"));
$first = true;
foreach($labels as $line) {
    if($first){
        $first = false;
    }else{
        if($line){
            [ $id, $label ] = explode(",", $line);
            $id_to_label[$id] = $label;
            $id_to_number[$id] = $id_count;
            $id_count++;
        }
    }
}
echo "Read " . count($id_to_label) . " labels";



Read 458913 labels

In [2]:
$TRAIN_PERC = 0.7;
$DEV_PERC   = 0.1;
$TEST_PERC  = 0.2;

$train_ids = array();    
$dev_ids = array();    
$test_ids = array();

mt_srand(1993);
$max = mt_getrandmax() * 1.0;

foreach($id_to_label as $id => $label) {
    $f = mt_rand() / $max;
    if($f < $TRAIN_PERC) {
        $train_ids[$id] = $label;
    }elseif($f < $TRAIN_PERC + $DEV_PERC){
        $dev_ids[$id] = $label;
    }else{
        $test_ids[$id] = $label;
    }
}
echo "Train: " . count($train_ids) . '<br>';
echo "Dev: "   . count($dev_ids)   . '<br>';
echo "Test: "  . count($test_ids);



Train: 321291 Dev: 45582 Test: 92040

In [3]:
$train_samples = [];
$dev_samples   = [];
$test_samples  = [];
$train_names = [];
$dev_names   = [];
$test_names  = [];
$train_labels  = [];
$dev_labels    = [];
$test_labels   = [];

$handle = fopen("data/train_data.csv", "r");
$line = fgets($handle); // header
$fnames = explode(",", $line);
array_shift($fnames); // ID
$date = array_shift($fnames); // date, SP_2
array_unshift($fnames, $date . "_y", $date . "_m", $date . "_d");

$line = fgets($handle);
$total_samples = 0;
echo date(DATE_RFC2822) . '<br>';
while ($line !== false) {
  $fields = explode(",", $line);
  $id = array_shift($fields);
  if(isset($id_to_label[$id])) {
     $row=[];
     $date = array_shift($fields);
     [$y, $m, $d] = explode("-", $date);
     $row[] = floatval($y);
     $row[] = floatval($m);
     $row[] = floatval($d);
     foreach($fields as $v) {
        if($v === ""){
           $v = NAN;
        }else{
           $v = floatval($v);
        }
        $row[] = $v;
     }
     if(isset($train_ids[$id])) {
         $train_samples[] = $row;
         $train_labels[]  = $id_to_label[$id];
         $train_names[]   = $id_to_number[$id];
     }elseif(isset($dev_ids[$id])){
         $dev_samples[] = $row;
         $dev_labels[]  = $id_to_label[$id];
         $dev_names[]   = $id_to_number[$id];
     }else{
         $test_samples[] = $row;
         $test_labels[]  = $id_to_label[$id];
         $test_names[]   = $id_to_number[$id];
     }
     $total_samples++;
     $line = fgets($handle);
  }
}
fclose($handle);

echo date(DATE_RFC2822) . '<br>';
echo "Read " . $total_samples . " samples" . '<br><br>';
echo "Train: " . count($train_samples) . '<br>';
echo "Dev: "   . count($dev_samples)   . '<br>';
echo "Test: "  . count($test_samples);


file_put_contents("work/training-data.bin", igbinary_serialize(array(
    "features" => $fnames,
    "train"    => [ $train_samples, $train_labels, $train_names ],
    "dev"      => [ $dev_samples,   $dev_labels,   $dev_names   ],
    "test"     => [ $test_samples,  $test_labels,  $test_names  ])), LOCK_EX);


Sat, 11 Jun 2022 00:52:58 -0700 Sat, 11 Jun 2022 00:57:00 -0700 Read 5531451 samples Train: 3873579 Dev: 548496 Test: 1109376